In [1]:
import os
import shutil

import session_processing_helper_c5v2 as helper
import utils_c5v2 as utils

import pandas as pd

In [ ]:
data_dir = '/Users/rebekahzhang/data/behavior_data'
period = '20240502-20241219'
exp = "exp2"
cohort = "cohort_7"
data_folder = os.path.join(data_dir, period, exp, cohort)
print(data_folder)

/Users/rebekahzhang/data/behavior_data/20250113-20250321/exp2/cohort_7


# Quality Control

## Check session folders have both meta and events

In [ ]:
missing_events, missing_meta, empty_meta, empty_events = helper.check_session_files(data_folder)
if not (missing_meta.empty and missing_events.empty and empty_meta.empty and empty_events.empty):
    print("\nFile check results:")
    if not missing_meta.empty:
        print("\nSessions missing meta files:")
        display(missing_meta)
    if not missing_events.empty:
        print("\nSessions missing events files:")
        display(missing_events)
    if not empty_meta.empty:
        print("\nSessions with empty meta files:")
        display(empty_meta)
    if not empty_events.empty:
        print("\nSessions with empty events files:")
        display(empty_events)
else:
    print("\nAll sessions have non-empty meta and events files.")

In [ ]:
utils.delete_folders(missing_meta.dir.tolist(), data_folder)

In [ ]:
utils.delete_folders(missing_events.dir.tolist(), data_folder)

In [ ]:
utils.delete_folders(empty_meta.dir.tolist(), data_folder)

In [ ]:
utils.delete_folders(empty_events.dir.tolist(), data_folder)

## Generate and save sessions log

generate session log using meta data from each session and add columns of basic info to each session

### re-run after every quality control steps

In [ ]:
sessions_all, sessions_training = helper.generate_session_logs(data_folder)
sessions_training.tail()

## Remove unwanted sessions
doesn't need to run when data folder is cleaned 
<br>
sessions_all needs to be regenerated after every cleaning step

### Remove test sessions
currently not needed cuz test sessions are not sorted into exp folders

In [ ]:
sessions_test = sessions_all.loc[sessions_all.mouse=='test']
utils.remove_sessions(sessions_test, data_folder)

### Remove crashed sessions
remove the sessions with ending_code==nan

In [ ]:
sessions_ended_nan = sessions_all[sessions_all.ending_code.isna()]
utils.remove_sessions(sessions_ended_nan, data_folder)

reomve sessions that crashed

In [ ]:
sessions_crashed = pd.DataFrame(columns=sessions_training.columns)
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)

for _, session_info in sessions_training.iterrows():
    try:
        events_path = utils.generate_events_path(data_folder, session_info)
        events = pd.read_csv(events_path, low_memory=False)
        session_end = events.loc[(events.key=='session') & (events.value==0)]

        if not len(session_end)==1:
            sessions_crashed = pd.concat([sessions_crashed, session_info.to_frame().T], ignore_index=True)
    
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

if len(problematic_sessions) > 0:
    print("cannot open: ")
    display(problematic_sessions)

if len(sessions_crashed) > 0:
    print("crashed sessions: ")
    display(sessions_crashed)
else:
    print("all sessions are perfect! woohoo!")

In [ ]:
utils.remove_sessions(sessions_crashed, data_folder)

### Check for short sessions

In [ ]:
short_threshold = 20
sessions_short = sessions_all[(sessions_all['total_trial'] < short_threshold) | sessions_all['total_trial'].isna()]
if len(sessions_short)>0:
    display(sessions_short)
else: 
    print('no short sessions to be checked!')

remove short sessions if needed

In [ ]:
utils.remove_sessions(sessions_short, data_folder)

# Process Events

load session log

In [ ]:
sessions_training = utils.load_data(os.path.join(data_folder, 'sessions_training.csv'))

In [ ]:
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)

for _, session_info in sessions_training.iterrows():
    try:
        events_processed_path = utils.generate_events_processed_path(data_folder, session_info)
        if os.path.isfile(events_processed_path):
            continue
        events = pd.read_csv(utils.generate_events_path(data_folder, session_info), low_memory=False)
        events = helper.process_events(session_info, events)
        events_processed = events.groupby('session_trial_num', group_keys=False).apply(helper.add_trial_time)
        events_processed.to_csv(events_processed_path)
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

if len(problematic_sessions) > 0:
    display(problematic_sessions)
else:
    print("all sessions are perfect! woohoo!")

# Data set curation 

In [ ]:
# deprecated rounds dict cuz we no longer run more than 1 session per day
# mouse_list = utils.generate_mouse_list(sessions_all)
# mouse_list = ['RZ047','RZ049','RZ050','RZ051','RZ052','RZ053','RZ054','RZ055','RZ056']
# mouse_list = ["RZ057", "RZ058", "RZ059", "RZ061", "RZ062", "RZ063", "RZ064",
#                      "RZ065", "RZ067", "RZ068", "RZ069", "RZ070", "RZ072"]
mouse_list = ["RZ057", "RZ058", "RZ059", "RZ061", "RZ062", "RZ063",
                     "RZ065", "RZ067", "RZ068", "RZ069", "RZ070"]
print(mouse_list)
sessions_by_date = sessions_training.groupby('date')

### Deal with missing sessions
this is not the proper way to deal with this. should have it populated with mean and variation

In [ ]:
no_missing_sessions = True
for date, data in sessions_by_date:
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) < 1:
            no_missing_sessions = False
            print(f"on {date}, {mouse} has missing sessions")
if no_missing_sessions:
    print("no missing sessions!")

In [ ]:
sessions_by_date.get_group('2025-02-11').sort_values('mouse')

#### duplicate if you are sussed out of having to redo this

In [ ]:
utils.backup(data_folder)

### Deal with multiple sessions

In [ ]:
days_to_stitch = []
mice_to_stitch = []
for date, data in sessions_by_date:
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) > 1:
            days_to_stitch.append(date)
            mice_to_stitch.append(mouse)
            print(f"on {date}, {mouse} has {len(mouse_by_date)} sessions")
if not days_to_stitch:
    print("no sessions to stitch!")

In [ ]:
# run it if session stitching is needed, nothing would happen otherwise
# has to run more than once if there are more than 2 sessions. fix it for the next round pls
if not days_to_stitch:
    print("no sessions to stitch!")
else:
    for d, m in zip(days_to_stitch, mice_to_stitch):
        day = sessions_by_date.get_group(d)
        sessions_to_stitch = day[day['mouse'] == m]

        session_1_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[0])
        session_2_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[1])

        if os.path.exists(session_1_dir) and os.path.exists(session_2_dir):
            session_1 = pd.read_csv(session_1_dir)
            session_2 = pd.read_csv(session_2_dir)
            stitched_session = helper.stitch_sessions(session_1, session_2) 
            #should change to stitch events. stitch sessions should be deleted. to follow nomanclature, all session should be renamed to events!!

            stitched_session.to_csv(session_1_dir, index=False)
            shutil.rmtree(os.path.join(data_folder, sessions_to_stitch.iloc[1].dir))
            print(f"{d} {m} session 2 deleted")
        else:
            print("one of the sessions do not exist")

In [ ]:
sessions_all, sessions_training = helper.generate_session_logs(data_folder)

sessions_training.tail()

# Finalize sessions log

In [ ]:
def correct_sessions_training(data_folder, save_log=True):
    _, sessions_training = helper.generate_session_logs(data_folder, save_logs=False)
    session_info_list = []
    for _, session_info in sessions_training.iterrows():
        events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info), low_memory=False)
        session_basics = helper.get_session_basics(events_processed)
        session_basics['dir'] = session_info['dir']
        session_info_list.append(session_basics)
    sessions_info = pd.DataFrame(session_info_list)
    corrected_sessions_training = pd.merge(sessions_training, sessions_info, on="dir")
    corrected_sessions_training = corrected_sessions_training.drop(columns=['index', 'total_reward', 'total_trial', 'total_reward'])
    corrected_sessions_training = corrected_sessions_training.groupby('mouse', group_keys=False).apply(helper.assign_session_numbers)
    if save_log:
        utils.save_as_csv(df=corrected_sessions_training, folder=data_folder, filename='sessions_training.csv')
    return corrected_sessions_training

In [ ]:
sessions_training = correct_sessions_training(data_folder)

# Analyze events

In [ ]:
sessions_training = utils.load_data(os.path.join(data_folder, 'sessions_training.csv'))

## Generate Trials

In [ ]:
# generate all trials based on events processed
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)
for _, session_info in sessions_training.iterrows():
    try: 
        trials_path = utils.generate_trials_path(data_folder, session_info)
        if os.path.isfile(trials_path):
            continue
        
        events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info))
        trials = helper.generate_trials(session_info, events_processed)

        trials.to_csv(trials_path)
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

if len(problematic_sessions) > 0:
    display(problematic_sessions)
else:
    print("all sessions are perfect! woohoo!")

## Analyze trials

In [ ]:
for _, session_info in sessions_training.iterrows():
    try:
        trials_analyzed_path = utils.generate_trials_analyzed_path(data_folder, session_info)
        if os.path.isfile(trials_analyzed_path):
            continue
        
        session_by_trial = utils.load_data(utils.generate_events_processed_path(data_folder, session_info)).groupby('session_trial_num')
        trials = utils.load_data(utils.generate_trials_path(data_folder, session_info))
        trials_data = helper.get_trial_data_df(session_by_trial)
        trials_analyzed = pd.merge(trials, trials_data, on='session_trial_num')
        trials_analyzed['group'] = session_info.group #assigning trial type manually
        trials_analyzed.to_csv(trials_analyzed_path)
    except:
        display(session_info)

debug debug debug

In [ ]:
# data_dir = '/Users/rebekahzhang/data/behavior_data'
# exp = "exp2"
# cohort = "c567"
# data_folder = os.path.join(data_dir, exp, cohort)
# sessions_training = pd.read_csv(os.path.join(data_folder, 'sessions_training.csv'), index_col=0)

# sessions_training_post_meta_change = sessions_training.loc[sessions_training.date < "2024-05-03"]

# for _, session_info in sessions_training_post_meta_change.iterrows():
#     try:
#         trials_analyzed_path = utils.generate_trials_analyzed_path(data_folder, session_info)
#         # if os.path.isfile(trials_analyzed_path):
#         #     continue
        
#         session_by_trial = utils.load_data(utils.generate_events_processed_path(data_folder, session_info)).groupby('session_trial_num')
#         trials = utils.load_data(utils.generate_trials_path(data_folder, session_info))
#         trials_data = helper.get_trial_data_df(session_by_trial)
#         trials_analyzed = pd.merge(trials, trials_data, on='session_trial_num')
#         trials_analyzed['group'] = session_info.group #assigning trial type manually
#         trials_analyzed.to_csv(trials_analyzed_path)
#     except:
#         display(session_info)